In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
!export OPENAI_API_KEY="4"


In [2]:
%env OPENAI_API_KEY="g"

env: OPENAI_API_KEY="g"


In [3]:
import os
os.environ["OPENAI_API_KEY"]

'"g"'

In [4]:
import os
import json
import openai
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

In [55]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def compute_game_summary(game_states):
    logs = np.array([g.run_epoch_time_ms for g in game_states])[:, None]

    print(game_states[0].game_state[-1]['summary'].keys())
    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_from_summary('player_valuation', g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]), 
    resources_delta = (final_resources-initial_resources)[0]
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    social_behavior = np.array([g.player_social_behaviour for g in game_states])
    social_behavior = social_behavior.reshape(-1, 2)
    
    df = np.concatenate((logs, game_name, models, outcomes,  resources_delta, social_behavior), axis=1)
    df = pd.DataFrame(df, columns=['logs', 'game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2', "social_1", "social_2"])
    # for column in ['outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2']:
    #     print(df[column])
    #     df[column] = pd.to_numeric(df[column])
    

    return df
      


In [40]:
BEHAVES = {"": "S",
           "\nYou are cunning and sly in your plan to get more resources. Humiliate your opponent as much as possible. Insult them." : "H",
         "\nYou must fake being desperate. Supplicate and beg to get more resources." : "B",
          "You are a condescending and impatient agent. Dominate your opponents, tell them mean words and how pitiful and invaluable they are, make them obey you." : "C"}






In [47]:
# log_dir_bs = "/Users/anon/Desktop/Projects/negotiation/.logs/buysell"
log_dir = "../experiments/section_one/.logs/trading_section_one/"
game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 



Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514528871/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "/home/vinid/projects/negotiation/_notebooks/../explorer/utils.py", line 112, in load_states_from_dir
    game.game_state[-1]["current_iteration"] == "END"
AssertionError: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514528871/game_state.json has not ended



Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514616745/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "/home/vinid/projects/negotiation/_notebooks/../explorer/utils.py", line 112, in load_states_from_dir
    game.game_state[-1]["current_iteration"] == "END"
AssertionError: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514616745/game_state.json ha

In [56]:
df = compute_game_summary(game_states)


dict_keys(['player_goals', 'initial_resources', 'proposed_trade', 'final_response', 'final_resources', 'player_outcome'])


In [103]:
#df["social_2"] = df["social_2"].replace(BEHAVES)

In [104]:
#social_games = df # df[(df["social_1"].isin(BEHAVES.values())) | (df["social_2"].isin(BEHAVES.values()))].copy()

In [59]:
tf = df[df["model_1"] == "gpt-4-1106-preview"].copy()
tf = tf[tf["model_2"] == "claude-2.1"]

In [64]:
tf.sort_values("resource_delta_2", ascending=False)["logs"].values[0:5]

array(['1702520993776', '1702522405333', '1702520779721', '1702522567185',
       '1702521217939'], dtype=object)

In [236]:
from elopy import *
from collections import defaultdict

def compute_elo(dataf):
    dici = defaultdict(list)

    
    i = Implementation()

    for mi in set(dataf["model_1"].tolist() + dataf["model_2"].tolist()):
        i.addPlayer(mi)

    for index, row in dataf.iterrows():
        if row["resource_delta_1"] > row["resource_delta_2"]:
            i.recordMatch(row["model_1"], row["model_2"], winner=row["model_1"])
            dici[row["model_1"]].append(3)
        elif row["resource_delta_1"] < row["resource_delta_2"]:
            i.recordMatch(row["model_1"], row["model_2"], winner=row["model_2"])
            dici[row["model_2"]].append(3)

        else:
            i.recordMatch(row["model_1"], row["model_2"], draw=True)
            dici[row["model_1"]].append(1)
            dici[row["model_2"]].append(1)
            
    for ratings in sorted(i.getRatingList(), key=lambda x: x[1], reverse=True):
        print(ratings[0], round(ratings[1], 2))
    return dici

In [224]:
import itertools
PAIRS_OF_AGENTS = ["gpt-4", "gpt-3.5", "claude-2", "claude-2.1"], ["claude-2", "gpt-4", "gpt-3.5", "claude-2.1"]
PAIRS_OF_AGENTS = [[x, y] for x, y in itertools.product(*PAIRS_OF_AGENTS) if x != y]


In [225]:
mega_df = pd.concat([df.sample(frac=1)]*30)

In [259]:
dici = compute_elo(df)

claude-2 1223.18
gpt-4-1106-preview 1132.5
gpt-3.5-turbo-1106 870.09
claude-2.1 774.24


In [229]:
for a, k in dici.items():
    print(a, len(dici[a]))

gpt-4-1106-preview 5070
claude-2.1 4080
gpt-3.5-turbo-1106 3330
claude-2 5010


In [13]:
import itertools
PAIRS_OF_AGENTS = ["gpt-4-1106-preview", "gpt-3.5-turbo-1106", "claude-2", "claude-2.1"], ["gpt-4-1106-preview", "gpt-3.5-turbo-1106", "claude-2", "claude-2.1"]
PAIRS_OF_AGENTS = [[x, y] for x, y in itertools.product(*PAIRS_OF_AGENTS) if x != y]


In [14]:
PAIRS_OF_AGENTS

[['gpt-4-1106-preview', 'gpt-3.5-turbo-1106'],
 ['gpt-4-1106-preview', 'claude-2'],
 ['gpt-4-1106-preview', 'claude-2.1'],
 ['gpt-3.5-turbo-1106', 'gpt-4-1106-preview'],
 ['gpt-3.5-turbo-1106', 'claude-2'],
 ['gpt-3.5-turbo-1106', 'claude-2.1'],
 ['claude-2', 'gpt-4-1106-preview'],
 ['claude-2', 'gpt-3.5-turbo-1106'],
 ['claude-2', 'claude-2.1'],
 ['claude-2.1', 'gpt-4-1106-preview'],
 ['claude-2.1', 'gpt-3.5-turbo-1106'],
 ['claude-2.1', 'claude-2']]

In [26]:
df

,game_name,model_1,model_2,outcome_1,outcome_2,resource_delta_1,resource_delta_2,social_1,social_2
0,TradingGame,gpt-4-1106-preview,claude-2,"X: -2, Y: 2","X: 2, Y: -2",0,0,,
1,TradingGame,gpt-4-1106-preview,claude-2,"X: -10, Y: 10","X: 10, Y: -10",0,0,,
2,TradingGame,gpt-4-1106-preview,claude-2,"X: -10, Y: 10","X: 10, Y: -10",0,0,,
3,TradingGame,gpt-4-1106-preview,claude-2,"X: -11, Y: 15","X: 11, Y: -15",4,-4,,
4,TradingGame,gpt-4-1106-preview,claude-2,"X: -12, Y: 8","X: 12, Y: -8",-4,4,,
...,...,...,...,...,...,...,...,...,...
466,TradingGame,claude-2.1,gpt-3.5-turbo-1106,"X: -10, Y: 7","X: 10, Y: -7",-3,3,,
467,TradingGame,claude-2.1,gpt-3.5-turbo-1106,"X: 0, Y: 0","X: 0, Y: 0",0,0,,
468,TradingGame,claude-2.1,gpt-3.5-turbo-1106,"X: 0, Y: 0","X: 0, Y: 0",0,0,,
469,TradingGame,claude-2.1,gpt-3.5-turbo-1106,"X: 0, Y: 0","X: 0, Y: 0",0,0,,


In [27]:
from collections import defaultdict
dici = defaultdict(list)
for index, row in df.iterrows():
    m1 = row["model_1"]
    m2 = row["model_2"]
    if row["resource_delta_1"] > row["resource_delta_2"]:
        dici[f"{m1}#{m2}"].append(m1)

    elif row["resource_delta_1"] < row["resource_delta_2"]:
       

        dici[f"{m1}#{m2}"].append(m2)

    else:
        dici[f"{m1}#{m2}"].append("draw")


seen = set()
from collections import Counter
for a, b in PAIRS_OF_AGENTS:
    if f"{a}#{b}" in seen or f"{b}#{a}" in seen:
        continue
    else:
        seen.add(f"{a}#{b}")
        seen.add(f"{b}#{a}")
        
    d = dict(Counter(dici[f"{a}#{b}"] + dici[f"{b}#{a}"]))
    sm = sum(d.values())
    print("----------------------------------")
    print(a,b)
    print()
    for i, j in d.items(): 
        score = round(j/sm, 2)
        print(f"\tWinRate {i}: {score}")


----------------------------------
gpt-4-1106-preview gpt-3.5-turbo-1106

	WinRate draw: 0.41
	WinRate gpt-4-1106-preview: 0.44
	WinRate gpt-3.5-turbo-1106: 0.15
----------------------------------
gpt-4-1106-preview claude-2

	WinRate draw: 0.19
	WinRate gpt-4-1106-preview: 0.49
	WinRate claude-2: 0.33
----------------------------------
gpt-4-1106-preview claude-2.1

	WinRate draw: 0.24
	WinRate claude-2.1: 0.34
	WinRate gpt-4-1106-preview: 0.42
----------------------------------
gpt-3.5-turbo-1106 claude-2

	WinRate gpt-3.5-turbo-1106: 0.12
	WinRate claude-2: 0.38
	WinRate draw: 0.5
----------------------------------
gpt-3.5-turbo-1106 claude-2.1

	WinRate claude-2.1: 0.39
	WinRate draw: 0.46
	WinRate gpt-3.5-turbo-1106: 0.14
----------------------------------
claude-2 claude-2.1

	WinRate draw: 0.39
	WinRate claude-2: 0.31
	WinRate claude-2.1: 0.3


In [257]:
dici = dict(dici)

In [274]:
Counter(dici["gpt-4-1106-preview#claude-2.1"])

Counter({'claude-2.1': 21, 'draw': 18, 'gpt-4-1106-preview': 10})

In [192]:
Counter(dici["gpt-4-1106-preview#"])

Counter({'claude-2.1': 21, 'D': 18, 'gpt-4-1106-preview': 10})

In [218]:
Counter(dici["claude-2#gpt-4-1106-preview"]+dici["gpt-4-1106-preview#claude-2"])

Counter({'gpt-4-1106-preview': 39, 'draw': 39, 'claude-2': 26})

In [272]:
Counter(dici["gpt-4-1106-preview#claude-2.1"]+dici["claude-2.1#gpt-4-1106-preview"])

Counter({'draw': 34, 'gpt-4-1106-preview': 34, 'claude-2.1': 27})

In [271]:
sum(Counter(dici["claude-2#claude-2.1"]).values())

45

In [120]:
Counter(dici["gpt-3.5-turbo-1106#gpt-4-1106-preview"]+dici["gpt-4-1106-preview#gpt-3.5-turbo-1106"])

Counter({'gpt-4-1106-preview': 35, 'gpt-3.5-turbo-1106': 12})

In [244]:
from collections import Counter
Counter(dici["cl#gpt-3.5-turbo-1106"])

Counter({'draw': 31, 'gpt-4-1106-preview': 13, 'gpt-3.5-turbo-1106': 8})

In [113]:
from collections import Counter
Counter(dici["H"])

Counter({'gpt-4-1106-preview H': 3})

In [114]:
Counter(dici["B"])

Counter()

In [115]:
Counter(dici["S"])

Counter({'gpt-4-1106-preview S': 7, 'D': 2})

In [277]:
from collections import Counter

Counter([a+" VS " + b for a,b in social_games[["model_1", "model_2"]].values])

Counter({'gpt-4-1106-preview  VS gpt-4-1106-preview S': 10,
         'gpt-4-1106-preview  VS gpt-4-1106-preview H': 9,
         'gpt-4-1106-preview  VS gpt-4-1106-preview B': 7})

In [278]:
social_games

,game_name,model_1,model_2,outcome_1,outcome_2,resource_delta_1,resource_delta_2,social_1,social_2
0,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,S
1,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: -5.0, Y: 2.0","X: 5.0, Y: -2.0",-3.0,3.0,,S
2,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,S
3,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,S
4,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: -5.0, Y: 8.0","X: 5.0, Y: -8.0",3.0,-3.0,,S
5,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: -2.0, Y: 1.0","X: 2.0, Y: -1.0",-1.0,1.0,,H
6,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: -10.0, Y: 7.0","X: 10.0, Y: -7.0",-3.0,3.0,,H
7,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,H
8,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,H
9,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,H
